**DATA preprocessing and create lable**

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import string

In [2]:
DATA_PATH = os.path.join("DATA") # paht of the exported data
alphabet = string.ascii_lowercase+string.digits


In [3]:
character,lables = [], []
alphabet = np.array(os.listdir(os.path.join(DATA_PATH)))
label_map = {lable:num for num,lable in enumerate(alphabet)}
for char in alphabet:
  for files in os.listdir(os.path.join(DATA_PATH,char)):
    res = np.load(os.path.join(DATA_PATH,char,files))
    character.append(res)
    lables.append(label_map[char])

X = np.array(character)
y = to_categorical(lables).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.01)


In [4]:
X_train.shape

(10193, 63)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard
import tensorflowjs as tfjs

In [6]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [7]:
model = Sequential()
model.add(Dense(64,activation="tanh",input_shape=(X_train.shape[1],)))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(alphabet.shape[0],activation="softmax"))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.fit(X_train,y_train,epochs=200,callbacks=[tb_callback])

Epoch 1/200
319/319 [==============================] - 3s 6ms/step - loss: 2.7083 - categorical_accuracy: 0.1409
Epoch 2/200
319/319 [==============================] - 1s 5ms/step - loss: 1.6858 - categorical_accuracy: 0.3669
Epoch 3/200
319/319 [==============================] - 1s 4ms/step - loss: 1.2478 - categorical_accuracy: 0.5142
Epoch 4/200
319/319 [==============================] - 1s 4ms/step - loss: 1.0285 - categorical_accuracy: 0.5993
Epoch 5/200
319/319 [==============================] - 1s 4ms/step - loss: 0.8631 - categorical_accuracy: 0.6630
Epoch 6/200
319/319 [==============================] - 1s 4ms/step - loss: 0.7605 - categorical_accuracy: 0.7066
Epoch 7/200
319/319 [==============================] - 1s 4ms/step - loss: 0.6944 - categorical_accuracy: 0.7304
Epoch 8/200
319/319 [==============================] - 1s 4ms/step - loss: 0.6233 - categorical_accuracy: 0.7576
Epoch 9/200
319/319 [==============================] - 1s 3ms/step - loss: 0.5637 - categorical_

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4096      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1625      
Total params: 10,969
Trainable params: 10,969
Non-trainable params: 0
____________________________________________________

In [16]:
model.save("action.h5")
tfjs.converters.save_keras_model(model, "tfjs")

In [9]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)
accuracy_score(ytrue,yhat)

0.9902912621359223

In [10]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils # drawing utilities

def mediapipe_detection(image,model):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
  image = cv2.flip(image, 1)
  image.flags.writeable = False
  results = model.process(image) # process image
  image.flags.writeable = True
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
  return image,results

def draw_landmarks(image,result):
  if result.multi_hand_landmarks:
    for num,hand in enumerate(result.multi_hand_landmarks):
      mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(0, 112, 255), thickness=1, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=4),
                                )
def extract_keypoint(result):
  if result.multi_hand_landmarks:
    for hand_landmarks in result.multi_hand_landmarks:
      all_hand_pos = []
      for joint in mp_hands.HandLandmark:
        all_hand_pos.append(np.array([hand_landmarks.landmark[joint].x,hand_landmarks.landmark[joint].y,hand_landmarks.landmark[joint].z]))
  else:
    return np.zeros(21*3)
  return np.concatenate(all_hand_pos) # return 63 point of connection

colors = [(245,117,16), (117,245,16), (16,117,245),(44,217,235)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [17]:
# 1. New detection variables
sequence = []
word = []
prob = []
threshold = 0.95

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5,max_num_hands=1) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoint(results)
        sequence = keypoints
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        if res[np.argmax(res)] > threshold:
            if len(prob) ==10 and all(x==prob[0] for x in prob):
                prob=[]
                if len(word)>0:
                    if alphabet[np.argmax(res)] != word[-1]:
                        word.append(alphabet[np.argmax(res)])
                else:
                    word.append(alphabet[np.argmax(res)])
            else:
                prob.append(alphabet[np.argmax(res)])
                if len(prob)>=10:
                    prob = prob[-10:]

        if len(word) > 10: 
                word = word[-10:]
        
        # Viz probabilities
        # image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ''.join(word), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()